In [4]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier

df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_8_features = [feature for feature, importance in features_rf_sorted[:7]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_8_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
# Define y as before
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=66)

# # Split the data into training and testing sets based on the specified index
# X_train = X_top8.iloc[:1720]  # Up to and including index 1719
# X_test = X_top8.iloc[1720:]   # From index 1720 to the end

# y_train = y.iloc[:1720]       # Corresponding y values for training
# y_test = y.iloc[1720:]        # Corresponding y values for testing
from tqdm.auto import tqdm

# 정밀도 점수를 저장할 리스트 생성
precision_results = []

# tqdm을 사용하여 random_state에 대한 반복 실행
for state in tqdm(range(1, 101), desc='Calculating Precision Scores'):
    lr_model = LogisticRegression(max_iter=10000, random_state=state)
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_test)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    
    # 결과를 리스트에 추가
    precision_results.append({'Random_State': state, 'Precision': precision})

# 리스트를 DataFrame으로 변환
precision_results_df = pd.DataFrame(precision_results)

# 결과 출력
print(precision_results_df)

# 추가: 가장 높은 정밀도를 갖는 random_state 확인
best_result = precision_results_df.loc[precision_results_df['Precision'].idxmax()]
print(f"Best Random State: {best_result['Random_State']}, Highest Precision: {best_result['Precision']:.4f}")

Calculating Precision Scores:   0%|          | 0/100 [00:00<?, ?it/s]

    Random_State  Precision
0              1   0.609475
1              2   0.609475
2              3   0.609475
3              4   0.609475
4              5   0.609475
5              6   0.609475
6              7   0.609475
7              8   0.609475
8              9   0.609475
9             10   0.609475
10            11   0.609475
11            12   0.609475
12            13   0.609475
13            14   0.609475
14            15   0.609475
15            16   0.609475
16            17   0.609475
17            18   0.609475
18            19   0.609475
19            20   0.609475
20            21   0.609475
21            22   0.609475
22            23   0.609475
23            24   0.609475
24            25   0.609475
25            26   0.609475
26            27   0.609475
27            28   0.609475
28            29   0.609475
29            30   0.609475
30            31   0.609475
31            32   0.609475
32            33   0.609475
33            34   0.609475
34            35   0